In [ ]:
from python_simulation_manager.rust_handler import RustExperiment, RustExperimentBuilder
from python_simulation_manager.output import ExperimentOutput
from typing import override
from pathlib import Path
import numpy as np

In [ ]:
class MyRustOutput(ExperimentOutput):
    def __init__(self, out_path):
        super().__init__(out_path)
        self.title   = None
        self.result1: list = []
        self.result2: list = []

    @override
    def parse_output(self, line_number: int, line: str):
        if line_number == 0:
            self.title = line.strip()
            return
        
        slines = line.split(",")
        self.result1.append(float(slines[0]))
        self.result2.append(float(slines[1]))
        

In [ ]:
class RustExperimentHandler:
    
    def __init__(self, results_dir: str, exp_name: str):
        self.proj_dir  = Path.cwd().parent
        self.builder   = RustExperimentBuilder(self.proj_dir, results_dir, exp_name)

    def create(self, monte_carlo_trials: dict[int, int], temperature: np.ndarray) -> RustExperiment:
        cargo_toml_path = self.proj_dir / "rust_example" / "Cargo.toml"
        
        self.builder.set_output_type(MyRustOutput)
        self.builder.add_static_parameter("temperature", temperature)
        self.builder.add_scaling_parameter("monte_carlo_trials", monte_carlo_trials)
        self.builder.set_scale_variable_names(["Lx","Ly"])
        self.builder.set_cargo_toml_path(cargo_toml_path)
        experiment = self.builder.build()
        return experiment
    
    def load(self, lengths: list[int]) -> RustExperiment:
        
        self.builder.set_output_type(MyRustOutput)
        self.builder.set_scale_variables(lengths)
        self.builder.set_scale_variable_names(["Lx","Ly"])
        return self.builder.build(load_only=True)




In [ ]:
result_folder = "results"
exp_folder    = "rust_build_experiment"

handler = RustExperimentHandler(result_folder, exp_folder)

monte_carlo_trials = {
    2: 1_000,
    3: 5_000,
    4: 10_000
}
temperature = np.arange(0, 10, 1, np.int32)
experiment  = handler.create(monte_carlo_trials, temperature)
experiment.write_parameter_files()

In [ ]:
experiment.are_parameter_files_available()

Helper function to give meaning to "scale_variables"

In [ ]:
def lengths(exp: RustExperiment) -> list[int]:
    return exp.get_scale_variables()

In [ ]:
for length in lengths(experiment):
    experiment.run(length, verbose=False)

In [ ]:
experiment.are_results_available()

Helpher function for type hinting

In [ ]:
def results(exp: RustExperiment) -> dict[int, MyRustOutput]:
    return exp.get_results()

In [ ]:
for (length, res) in results(experiment).items():
    
    print(f"For length = {length}:")
    print(f"info: \"{res.title}\"")
    print(f"> result1 = {res.result1}")
    print(f"> result2 = {res.result2}")
    print("")
    
    assert type(res.result1) == np.ndarray, "Lists are cast to np array automatically!"
    assert type(res.result2) == np.ndarray, "Lists are cast to np array automatically!"

Load from file:

In [ ]:
result_folder = "test_folder"
exp_folder    = "rust_experiment"
lengths       = [2,3,4]

exp_loader     = RustExperimentHandler(result_folder, exp_folder)
exp_from_file  = exp_loader.load(lengths) 

In [ ]:
exp_from_file.are_results_available()

In [ ]:
for (length, res) in  results(exp_from_file).items():
    res: MyRustOutput
    
    print(f"For length = {length}:")
    print(f"info: \"{res.title}\"")
    print(f"> result1 = {res.result1}")
    print(f"> result2 = {res.result2}")
    print("")